<a href="https://colab.research.google.com/github/jj2883/Applied_DL_COMS4995/blob/master/App_DL_HW5_part1_jj2883.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow-gpu==2.0.0-alpha0

In [2]:
!pip install sacrebleu # https://github.com/mjpost/sacreBLEU


In [0]:

import numpy as np
import re
import sacrebleu
import tensorflow as tf
import time
import unicodedata

# Preparing the Dataset

I have uploaded the provided spa.txt file onto the google drive, and downloaded it into the Colab. Then, I randomly shuffled them to get variety of sentences. 

In [4]:
#mount google drive
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls "/content/drive/My Drive/App_DL_hw5/"

spa.txt


In [0]:
sentences_dummy = open('/content/drive/My Drive/App_DL_hw5/spa.txt','r').readlines()

In [7]:
print(len(sentences_dummy))
print(sentences_dummy[5])

118964
Run!	¡Corre!



## Randomly shufflel sentences and pick 2000 sentences

In [8]:
from random import shuffle
shuffle(sentences_dummy)
print(sentences_dummy[5])


Follow him.	Síguele.



In [9]:
sentences = sentences_dummy[1000:3000]
print(len(sentences))

2000


In [0]:
data = [i.strip('\n').split('\t') for i in sentences]


In [11]:
print(data[0])

['This looks good.', 'Esto va bien.']


In [0]:
def preprocess(s):
  # for details, see https://www.tensorflow.org/alpha/tutorials/sequences/nmt_with_attention
  s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
  s = s.strip()
  s = '<start> ' + s + ' <end>'
  return s

In [0]:
sentences=data

In [14]:
print("Original:", sentences[0])
sentences = [(preprocess(source), preprocess(target)) for (source, target) in sentences]
print("Preprocessed:", sentences[0])

Original: ['This looks good.', 'Esto va bien.']
Preprocessed: ('<start> This looks good . <end>', '<start> Esto va bien . <end>')


In [0]:
source_sentences, target_sentences = list(zip(*sentences))


In [16]:
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_tokenizer.fit_on_texts(source_sentences)
source_data = source_tokenizer.texts_to_sequences(source_sentences)
print("Sequence:", source_data[0])
source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
print("Padded:", source_data[0])

Sequence: [1, 22, 277, 82, 3, 2]
Padded: [  1  22 277  82   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [0]:
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
target_tokenizer.fit_on_texts(target_sentences)
target_data = target_tokenizer.texts_to_sequences(target_sentences) 
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')

In [18]:
# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(target_data.shape)
target_labels[:,0:target_data.shape[1] -1] = target_data[:,1:]

print("Target sequence", target_data[0])
print("Target label", target_labels[0])

Target sequence [  1  55 120  56   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Target label [ 55. 120.  56.   3.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [19]:
source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1
print(source_vocab_size)
print(target_vocab_size)

2136
2878


In [20]:
def decode(encoded, tokenizer):
  for number in encoded:
    if number !=0:
      print ("%d -> %s" % (number, tokenizer.index_word[number]))
      
decode(source_data[0], source_tokenizer)

1 -> <start>
22 -> this
277 -> looks
82 -> good
3 -> .
2 -> <end>


In [0]:
batch_size = 5
dataset = tf.data.Dataset.from_tensor_slices((source_data, target_data, target_labels)).batch(batch_size)

In [22]:

example_batch = next(iter(dataset))
source, target, taget_labels = example_batch
print("Shapes:", source.shape, target.shape, taget_labels.shape)

Shapes: (5, 28) (5, 28) (5, 28)


In [0]:
embedding_size = 32
rnn_size = 64

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self):
    super(Encoder, self).__init__()
    
    self.embedding = tf.keras.layers.Embedding(source_vocab_size,
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)        
    return output, state
  
  def init_state(self, batch_size):
    return tf.zeros((batch_size, rnn_size))

In [25]:

# Create a batch of one sentence
ex_sentence = tf.expand_dims(source_data[0], axis=0)
ex_translation = tf.expand_dims(target_data[0], axis=0)
ex_labels = tf.expand_dims(target_labels[0], axis=0)
print(ex_sentence.shape)

encoder = Encoder()
hidden_state = encoder.init_state(batch_size=1)
print(hidden_state.shape)

output, hidden_state = encoder(ex_sentence, hidden_state)
print(output.shape)

(1, 28)
(1, 64)
(1, 28, 64)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, 
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)

    self.dense = tf.keras.layers.Dense(target_vocab_size)


  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    logits = self.dense(output)
    return logits, state

In [27]:
decoder = Decoder()
decoder_output, decoder_state = decoder(ex_labels, hidden_state)
print(decoder_output.shape)

(1, 28, 2878)


In [28]:
crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def calc_loss(targets, logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  mask = tf.cast(mask, dtype=tf.int64)
  return crossentropy(targets, logits, sample_weight=mask)

print("Loss", calc_loss(ex_labels, decoder_output))

Loss tf.Tensor(1.4219977, shape=(), dtype=float32)


In [0]:
def translate(idx=None):
  
    if idx == None: 
      idx = np.random.choice(len(sentences))
    
    input_sent = source_data[idx]
    input_sent = tf.expand_dims(input_sent, axis=0)
    
    hidden_state = encoder.init_state(batch_size=1)
    output, hidden_state = encoder(input_sent, hidden_state)
    
    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []
    
    decoder_state = hidden_state

    while True:
      
        decoder_output, decoder_state = decoder(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break
          
    translation = ' '.join(out_words)    
    return sentences[idx][0], sentences[idx][1], translation

In [30]:
input_sent, target_sent, translation = translate()
print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Input: <start> Does he go to work by bus ? <end>
Target: <start> ¿ El va al trabajo en autobus ? <end>
Translation: alla use doce patinar ustedes quejas respecto amistad comer metros tecnica esperabamos perezoso nuestros dejando perdido rapidamente traten supuestamente estaciono



In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function # remove this annotation when debugging
def train_step(source_seq, target_seq, target_labels, initial_state):
  
  with tf.GradientTape() as tape:
    encoder_output, encoder_state = encoder(source_seq, initial_state)
    logits, decoder_state = decoder(target_seq, encoder_state)
    loss = calc_loss(target_labels, logits)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

# Part 1: Train model to translate from ENGLISH to SPANISH

In [33]:

EPOCHS = 300

for epoch in range(EPOCHS):
    start = time.time()
  
    en_initial_states = encoder.init_state(batch_size)
    
    for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
      loss = train_step(source_seq, target_seq, target_labels, en_initial_states)
      elapsed = time.time() - start
    
    if epoch % 10 == 0:
      print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
      input_sent, target_sent, translation = translate()
      print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Epoch #0, Loss 1.4839, Time 5.04 sec
Input: <start> We re going out for lunch . Why don t you come along ? <end>
Target: <start> Vamos a salir a almorzar . ¿ Por que no vienes ? <end>
Translation: ¿ no que . <end>

Epoch #10, Loss 0.9844, Time 2.99 sec
Input: <start> She majored in economics . <end>
Target: <start> Ella se especializo en economia . <end>
Translation: ¿ que no se gusta en la ciudad . <end>

Epoch #20, Loss 0.7291, Time 2.97 sec
Input: <start> I ll kill him . <end>
Target: <start> Lo matare . <end>
Translation: me gusta una poco de mary . <end>

Epoch #30, Loss 0.5208, Time 3.22 sec
Input: <start> I don t have money . <end>
Target: <start> No tengo un centimo . <end>
Translation: no hay nada que mary se entristecio a continuacion . <end>

Epoch #40, Loss 0.3884, Time 2.96 sec
Input: <start> If Tom had asked me for some money , I d have given it to him . <end>
Target: <start> Si Tom me hubiese pedido algo de dinero se lo habria dado . <end>
Translation: si estas profesore

In [34]:

references, hypotheses = [], []

for i in range(len(sentences)):
  input_sent, target_sent, translation = translate()
  references.append(target_sent)
  hypotheses.append("<start> " + translation)
  
results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
print(results)

BLEU(score=66.71949558305275, counts=[15576, 11216, 8772, 6857], totals=[18340, 16340, 14340, 12340], precisions=[84.92911668484187, 68.64137086903305, 61.17154811715481, 55.56726094003241], bp=1.0, sys_len=18340, ref_len=18323)


# Part2: Preparing data for Spanish to English Translation

In [0]:
sentences=data

In [36]:
print("Original:", sentences[0])
sentences = [(preprocess(target), preprocess(source)) for (source, target) in sentences]
print("Preprocessed:", sentences[0])

Original: ['This looks good.', 'Esto va bien.']
Preprocessed: ('<start> Esto va bien . <end>', '<start> This looks good . <end>')


In [0]:
source_sentences, target_sentences = list(zip(*sentences))


In [38]:
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_tokenizer.fit_on_texts(source_sentences)
source_data = source_tokenizer.texts_to_sequences(source_sentences)
print("Sequence:", source_data[0])
source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
print("Padded:", source_data[0])

Sequence: [1, 55, 120, 56, 3, 2]
Padded: [  1  55 120  56   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [39]:
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
target_tokenizer.fit_on_texts(target_sentences)
target_data = target_tokenizer.texts_to_sequences(target_sentences)
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')
print("Padded:", target_data[0])

Padded: [  1  22 277  82   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [40]:
# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(target_data.shape)
target_labels[:,0:target_data.shape[1] -1] = target_data[:,1:]

print("Target sequence", target_data[0])
print("Target label", target_labels[0])

Target sequence [  1  22 277  82   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Target label [ 22. 277.  82.   3.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [41]:
source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1
print(source_vocab_size)
print(target_vocab_size)

2878
2136


In [42]:
def decode(encoded, tokenizer):
  for number in encoded:
    if number !=0:
      print ("%d -> %s" % (number, tokenizer.index_word[number]))
      
decode(source_data[0], source_tokenizer)

1 -> <start>
55 -> esto
120 -> va
56 -> bien
3 -> .
2 -> <end>


In [0]:
batch_size = 5
dataset = tf.data.Dataset.from_tensor_slices((source_data, target_data, target_labels)).batch(batch_size)

In [44]:

example_batch = next(iter(dataset))
source, target, taget_labels = example_batch
print("Shapes:", source.shape, target.shape, taget_labels.shape)

Shapes: (5, 28) (5, 28) (5, 28)


In [0]:
embedding_size = 32
rnn_size = 64

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self):
    super(Encoder, self).__init__()
    
    self.embedding = tf.keras.layers.Embedding(source_vocab_size,
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)        
    return output, state
  
  def init_state(self, batch_size):
    return tf.zeros((batch_size, rnn_size))

In [47]:

# Create a batch of one sentence
ex_sentence = tf.expand_dims(source_data[0], axis=0)
ex_translation = tf.expand_dims(target_data[0], axis=0)
ex_labels = tf.expand_dims(target_labels[0], axis=0)
print(ex_sentence.shape)

encoder2 = Encoder()
hidden_state = encoder2.init_state(batch_size=1)
print(hidden_state.shape)

output, hidden_state = encoder2(ex_sentence, hidden_state)
print(output.shape)

(1, 28)
(1, 64)
(1, 28, 64)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, 
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)

    self.dense = tf.keras.layers.Dense(target_vocab_size)


  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    logits = self.dense(output)
    return logits, state

In [49]:
decoder2 = Decoder()
decoder_output, decoder_state = decoder2(ex_labels, hidden_state)
print(decoder_output.shape)

(1, 28, 2136)


In [50]:
crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def calc_loss(targets, logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  mask = tf.cast(mask, dtype=tf.int64)
  return crossentropy(targets, logits, sample_weight=mask)

print("Loss", calc_loss(ex_labels, decoder_output))

Loss tf.Tensor(1.3692524, shape=(), dtype=float32)


In [0]:
def translate_spa_eng(idx=None):
  
    if idx == None: 
      idx = np.random.choice(len(sentences))
    
    input_sent = source_data[idx]
    input_sent = tf.expand_dims(input_sent, axis=0)
    
    hidden_state = encoder2.init_state(batch_size=1)
    output, hidden_state = encoder2(input_sent, hidden_state)
    
    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []
    
    decoder_state = hidden_state

    while True:
      
        decoder_output, decoder_state = decoder2(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break
          
    translation = ' '.join(out_words)    
    return sentences[idx][0], sentences[idx][1], translation

In [52]:
input_sent, target_sent, translation = translate_spa_eng()
print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Input: <start> Se que has estado intentando ayudar . <end>
Target: <start> I know you ve been trying to help . <end>
Translation: colleagues red clearness sang college dangerous reprimanded nuts needed rain oiled obsessed four gave paparazzi faintest singer positive hates hospitals



In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function # remove this annotation when debugging
def train_step(source_seq, target_seq, target_labels, initial_state):
  
  with tf.GradientTape() as tape:
    encoder_output, encoder_state = encoder2(source_seq, initial_state)
    logits, decoder_state = decoder2(target_seq, encoder_state)
    loss = calc_loss(target_labels, logits)

  variables = encoder2.trainable_variables + decoder2.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

# Part 2: Train model to translate from SPANISH to ENGLISH

In [55]:

EPOCHS = 300

for epoch in range(EPOCHS):
    start = time.time()
  
    en_initial_states = encoder2.init_state(batch_size)
    
    for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
      loss = train_step(source_seq, target_seq, target_labels, en_initial_states)
      elapsed = time.time() - start
    
    if epoch % 10 == 0:
      print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
      input_sent, target_sent, translation = translate_spa_eng()
      print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Epoch #0, Loss 1.2448, Time 4.84 sec
Input: <start> El sabe leer bien . <end>
Target: <start> He can read well . <end>
Translation: i is the . <end>

Epoch #10, Loss 0.8066, Time 2.88 sec
Input: <start> Me gusta mucho la comida francesa . <end>
Target: <start> I like French food very much . <end>
Translation: i m tired of the street of the day . <end>

Epoch #20, Loss 0.6183, Time 2.86 sec
Input: <start> La gente se burlo de Tom . <end>
Target: <start> People laughed at Tom . <end>
Translation: i m tired of the worst . <end>

Epoch #30, Loss 0.4454, Time 3.40 sec
Input: <start> Por favor espera tu turno . <end>
Target: <start> Please wait until your turn . <end>
Translation: please say that he s innocent . <end>

Epoch #40, Loss 0.3313, Time 2.86 sec
Input: <start> ¿ Les dijiste a todos donde sera la siguiente reunion ? <end>
Target: <start> Have you told everyone where the meeting will be ? <end>
Translation: do you want to go ? <end>

Epoch #50, Loss 0.2609, Time 2.81 sec
Input: <sta

# Part 3: Translate English to Spanish first, then translate Spanish back to English

## First Translate English to Spanish

In [0]:
sentences=data

In [98]:
print("Original:", sentences[0])
sentences = [(preprocess(source), preprocess(target)) for (source, target) in sentences]
print("Preprocessed:", sentences[0])

Original: ['This looks good.', 'Esto va bien.']
Preprocessed: ('<start> This looks good . <end>', '<start> Esto va bien . <end>')


In [0]:
source_sentences, target_sentences = list(zip(*sentences))


In [100]:
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_tokenizer.fit_on_texts(source_sentences)
source_data = source_tokenizer.texts_to_sequences(source_sentences)
print("Sequence:", source_data[0])
source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
print("Padded:", source_data[0])

Sequence: [1, 22, 277, 82, 3, 2]
Padded: [  1  22 277  82   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [0]:
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
target_tokenizer.fit_on_texts(target_sentences)
target_data = target_tokenizer.texts_to_sequences(target_sentences) 
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')

In [102]:
# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(target_data.shape)
target_labels[:,0:target_data.shape[1] -1] = target_data[:,1:]

print("Target sequence", target_data[0])
print("Target label", target_labels[0])

Target sequence [  1  55 120  56   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Target label [ 55. 120.  56.   3.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [0]:
source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1


In [0]:
# Create a batch of one sentence
ex_sentence = tf.expand_dims(source_data[0], axis=0)
ex_translation = tf.expand_dims(target_data[0], axis=0)
ex_labels = tf.expand_dims(target_labels[0], axis=0)


### Store translated sentences into 'eng_to_spa'

In [109]:
eng_to_spa, references = [], []

for i in range(len(sentences)):
  input_sent, target_sent, translation = translate()
  references.append(input_sent)
  #print(input_sent)
  eng_to_spa.append("<start> " + translation)
  print(translation)

tom tiene problemas para pararse en una pierna . <end>
el hombre hace preguntas tontas . <end>
ayer hubo un terremoto . <end>
esta palabra no se utiliza asi . <end>
¿ que pasas haciendo la mayor parte del tiempo ? <end>
yo solo se que tom no es feliz . <end>
el conduce un camion . <end>
tomas volvera pronto te he visto . <end>
no le digas nada a nadie . <end>
tom quiere ser abrazado . <end>
todavia es demasiado pronto para levantarse . <end>
el valle era de veinte millas de ancho . <end>
vista de lejos , se ve como una pelota . <end>
adivina que tengo en la mano . <end>
los fantasmas existen . <end>
¿ como se ve tu auto ? <end>
en cuanto a la remuneracion que vas a percibir , ¿ es este un buen trabajo ? <end>
sabe quien es . <end>
tenemos una ventaja . <end>
tom no nos va a decir nada . <end>
es muy dificil realizar esa empresa . <end>
no quise despertarte . <end>
tom tiene que ser canadiense . <end>
no voy a usar ese vestido . <end>
¿ que sabe tom sobre mi ? <end>
tenemos un monton de

In [110]:
print(eng_to_spa)

['<start> tom tiene problemas para pararse en una pierna . <end>', '<start> el hombre hace preguntas tontas . <end>', '<start> ayer hubo un terremoto . <end>', '<start> esta palabra no se utiliza asi . <end>', '<start> ¿ que pasas haciendo la mayor parte del tiempo ? <end>', '<start> yo solo se que tom no es feliz . <end>', '<start> el conduce un camion . <end>', '<start> tomas volvera pronto te he visto . <end>', '<start> no le digas nada a nadie . <end>', '<start> tom quiere ser abrazado . <end>', '<start> todavia es demasiado pronto para levantarse . <end>', '<start> el valle era de veinte millas de ancho . <end>', '<start> vista de lejos , se ve como una pelota . <end>', '<start> adivina que tengo en la mano . <end>', '<start> los fantasmas existen . <end>', '<start> ¿ como se ve tu auto ? <end>', '<start> en cuanto a la remuneracion que vas a percibir , ¿ es este un buen trabajo ? <end>', '<start> sabe quien es . <end>', '<start> tenemos una ventaja . <end>', '<start> tom no nos v

## Translate Translated Spanish back to English

In [0]:
sentences=data

In [112]:
print("Original:", sentences[0])
sentences = [(preprocess(target), preprocess(source)) for (source, target) in sentences]
print("Preprocessed:", sentences[0])

Original: ['This looks good.', 'Esto va bien.']
Preprocessed: ('<start> Esto va bien . <end>', '<start> This looks good . <end>')


### Store translated sentences into 'source_sentences'

In [0]:
source_sentences, target_sentences = list(zip(*sentences))


source_sentences = eng_to_spa

In [115]:
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_tokenizer.fit_on_texts(source_sentences)
source_data = source_tokenizer.texts_to_sequences(source_sentences)
print("Sequence:", source_data[0])
source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
print("Padded:", source_data[0])

Sequence: [1, 7, 44, 136, 31, 567, 13, 19, 568, 3, 2]
Padded: [  1   7  44 136  31 567  13  19 568   3   2   0   0   0   0   0   0   0
   0   0   0]


In [0]:
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
target_tokenizer.fit_on_texts(target_sentences)
target_data = target_tokenizer.texts_to_sequences(target_sentences) 
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')

In [117]:
# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(target_data.shape)
target_labels[:,0:target_data.shape[1] -1] = target_data[:,1:]

print("Target sequence", target_data[0])
print("Target label", target_labels[0])

Target sequence [  1  22 277  82   3   2   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
Target label [ 22. 277.  82.   3.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [0]:
source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1


In [0]:
# Create a batch of one sentence
ex_sentence = tf.expand_dims(source_data[0], axis=0)
ex_translation = tf.expand_dims(target_data[0], axis=0)
ex_labels = tf.expand_dims(target_labels[0], axis=0)


## Translate Spanish to English and store the sentences into 'eng_to_spa'

In [123]:
spa_to_eng = []

for i in range(len(sentences)):
  input_sent, target_sent, translation = translate_spa_eng()
  print(translation)
  spa_to_eng.append("<start> " + translation)

tom never lived in boston . <end>
tom left the house to his son . <end>
tom s a classic . <end>
you should conform to the rules . <end>
he bought a lot in a box . <end>
in a is strange ? <end>
did you know what he is ? <end>
my name isn t be a coincidence . <end>
tom will play afraid and you come . <end>
i think it s unlikely that i ll be able to pass my driving test . <end>
you don t need to come . <end>
this looks good . <end>
within of the students were from asia out . <end>
did you know what he is ? <end>
year is a little cold . <end>
he can t buy it . <end>
is it your birthday today ? <end>
let s see what he is proud of this is . <end>
there was tom with him . <end>
it is said that time heals all the wounds . <end>
i won t have a son named tom . <end>
no one will be fired . <end>
is there your second going to the concert ? <end>
tom never knew his parents . <end>
the party in power is corrupt , but the opposition is little better . <end>
he made his way . <end>
tom said he needed 

### Print Input Sentences in English

In [121]:

print(references)

['<start> Tom has trouble standing on one leg . <end>', '<start> The man hit me on the head . <end>', '<start> Yesterday there was an earthquake . <end>', '<start> This word isn t used like that . <end>', '<start> What do you spend a majority of your time doing ? <end>', '<start> I only know that Tom isn t happy . <end>', '<start> He drives a truck . <end>', '<start> Tom will soon be back . <end>', '<start> Don t say anything to anybody . <end>', '<start> Tom wants to be hugged . <end>', '<start> It s still too early to get up . <end>', '<start> The valley was twenty miles wide . <end>', '<start> Seen from a distance , it looks like a ball . <end>', '<start> Guess what I m holding in my hand . <end>', '<start> Ghosts exist . <end>', '<start> What does your car look like ? <end>', '<start> In terms of the pay you will get , is this a good job ? <end>', '<start> He knows who she is . <end>', '<start> We have one advantage . <end>', '<start> Tom won t tell us anything . <end>', '<start> I

### Print Sentences Translated Twice in English

In [124]:
print(spa_to_eng)

['<start> tom never lived in boston . <end>', '<start> tom left the house to his son . <end>', '<start> tom s a classic . <end>', '<start> you should conform to the rules . <end>', '<start> he bought a lot in a box . <end>', '<start> in a is strange ? <end>', '<start> did you know what he is ? <end>', '<start> my name isn t be a coincidence . <end>', '<start> tom will play afraid and you come . <end>', '<start> i think it s unlikely that i ll be able to pass my driving test . <end>', '<start> you don t need to come . <end>', '<start> this looks good . <end>', '<start> within of the students were from asia out . <end>', '<start> did you know what he is ? <end>', '<start> year is a little cold . <end>', '<start> he can t buy it . <end>', '<start> is it your birthday today ? <end>', '<start> let s see what he is proud of this is . <end>', '<start> there was tom with him . <end>', '<start> it is said that time heals all the wounds . <end>', '<start> i won t have a son named tom . <end>', '

## BLEU Score

In [125]:
results = sacrebleu.raw_corpus_bleu(spa_to_eng, [references])
print(results)

BLEU(score=0.6877028693598619, counts=[6101, 1610, 9, 2], totals=[19917, 17917, 15917, 13917], precisions=[30.632123311743737, 8.985879332477536, 0.056543318464534774, 0.014370913271538407], bp=1.0, sys_len=19917, ref_len=19225)


# I got BLEU Score of 0.6877!